In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,time,pickle
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from joblib import dump, load
from utils import data_process, visualize
from utils.utils import train, test, summary, setup_seed
from utils.models import Att_CNN, CNN, TFEQ, CRNN, M_CRNN

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [2]:
data = np.load("/home/yaso/EQ_Place/code/data/data_train.npy", allow_pickle=True)
x_train = data.item().get("x_train")
y_train = data.item().get("y_train")
data = np.load("/home/yaso/EQ_Place/code/data/data_test.npy", allow_pickle=True)
x_test = data.item().get("x_test")
y_test = data.item().get("y_test")
# x_train=x_train.transpose((0,2,1))
# x_test=x_test.transpose((0,2,1))


x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()
traindata = TensorDataset(x_train, y_train)
testdata = TensorDataset(x_test, y_test)

train_loader = DataLoader(traindata, batch_size=128, shuffle=True)
test_loader = DataLoader(testdata, batch_size=256, shuffle=True)

In [3]:
class TFEQ_v2(nn.Module):
    def __init__(self, channel=1, dmodel=64, nhead=8, dim_feedforward=32, num_layers=1, time_in=200):
        super(TFEQ_v2, self).__init__()
        self.input_embedding = nn.Linear(channel, dmodel)
        self.temporal_encoder_layer = nn.TransformerEncoderLayer(dmodel, nhead, dim_feedforward)
        self.temporal_encoder = nn.TransformerEncoder(self.temporal_encoder_layer, num_layers)
        self.temporal_pos = torch.arange(0, time_in).cuda()
        self.temporal_pe = nn.Embedding(time_in, dmodel)
        self.ln=nn.LayerNorm((time_in, dmodel))
        self.Feed1 = nn.Conv1d(in_channels=dmodel, out_channels=dmodel//2, kernel_size=1, stride=1)
        self.Feed2 = nn.Conv1d(in_channels=dmodel//2, out_channels=channel, kernel_size=1, stride=1)
        self.fla = nn.Flatten()
        self.dp = nn.Dropout(p=0.5)
        
        self.rnn = nn.RNN(input_size=time_in*channel, hidden_size=100)
        
        
        self.fc = nn.Linear(100, 2)

    def forward(self, x):
        b, t,  c = x.shape  # [B, 200, 3]
        x = self.input_embedding(x)  # [B, 200,  64]
        t_pe = self.temporal_pe(self.temporal_pos).expand(b, t, -1)
        x = x + t_pe
        x = self.temporal_encoder(x)
        x= self.ln(x)
        x = x.view(-1, x.shape[2], x.shape[1])
        x = F.relu(self.Feed1(x))
        x = F.relu(self.Feed2(x))
        x = self.fla(x)
        x = x.view(-1, 1, x.shape[-1])
        x, _ = self.rnn(x)
        
        x = self.fla(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


In [4]:
%%time
setup_seed(42)
# model = CRNN().cuda()
model=TFEQ_v2(channel=3, time_in=200).cuda()
L = []
test_hist = []
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 200 + 1):
    start = time.time()
    loss = train(model, train_loader, optimizer, epoch)
    L.append(loss)
    print("time {:.1f} sec:".format(time.time() - start))
    acc = test(model, test_loader)
    test_hist.append(acc)

Train Epoch: 1, Average Loss: 0.3616
time 1.8 sec:
Test set: Average loss: 0.3538, Accuracy: 1837/2070 (88.74%)

Train Epoch: 2, Average Loss: 0.3519
time 1.5 sec:
Test set: Average loss: 0.3518, Accuracy: 1837/2070 (88.74%)

Train Epoch: 3, Average Loss: 0.3521
time 1.5 sec:
Test set: Average loss: 0.3519, Accuracy: 1837/2070 (88.74%)

Train Epoch: 4, Average Loss: 0.3507
time 1.5 sec:
Test set: Average loss: 0.3521, Accuracy: 1837/2070 (88.74%)

Train Epoch: 5, Average Loss: 0.3512
time 1.5 sec:
Test set: Average loss: 0.3528, Accuracy: 1837/2070 (88.74%)

Train Epoch: 6, Average Loss: 0.3520
time 1.3 sec:
Test set: Average loss: 0.3521, Accuracy: 1837/2070 (88.74%)

Train Epoch: 7, Average Loss: 0.3513
time 1.3 sec:
Test set: Average loss: 0.3543, Accuracy: 1837/2070 (88.74%)

Train Epoch: 8, Average Loss: 0.3520
time 1.3 sec:
Test set: Average loss: 0.3532, Accuracy: 1837/2070 (88.74%)

Train Epoch: 9, Average Loss: 0.3510
time 1.4 sec:
Test set: Average loss: 0.3527, Accuracy: 183

Train Epoch: 73, Average Loss: 0.0375
time 1.5 sec:
Test set: Average loss: 0.0236, Accuracy: 2049/2070 (98.99%)

Train Epoch: 74, Average Loss: 0.0582
time 1.5 sec:
Test set: Average loss: 0.0501, Accuracy: 2029/2070 (98.02%)

Train Epoch: 75, Average Loss: 0.0459
time 1.5 sec:
Test set: Average loss: 0.0768, Accuracy: 2027/2070 (97.92%)

Train Epoch: 76, Average Loss: 0.0507
time 1.5 sec:
Test set: Average loss: 0.0235, Accuracy: 2051/2070 (99.08%)

Train Epoch: 77, Average Loss: 0.0457
time 1.4 sec:
Test set: Average loss: 0.0444, Accuracy: 2037/2070 (98.41%)

Train Epoch: 78, Average Loss: 0.0328
time 1.4 sec:
Test set: Average loss: 0.0264, Accuracy: 2048/2070 (98.94%)

Train Epoch: 79, Average Loss: 0.0375
time 1.4 sec:
Test set: Average loss: 0.0247, Accuracy: 2052/2070 (99.13%)

Train Epoch: 80, Average Loss: 0.0419
time 1.5 sec:
Test set: Average loss: 0.0259, Accuracy: 2048/2070 (98.94%)

Train Epoch: 81, Average Loss: 0.0325
time 1.5 sec:
Test set: Average loss: 0.0239, Accu

Train Epoch: 145, Average Loss: 0.0191
time 1.6 sec:
Test set: Average loss: 0.0187, Accuracy: 2054/2070 (99.23%)

Train Epoch: 146, Average Loss: 0.0176
time 1.5 sec:
Test set: Average loss: 0.0283, Accuracy: 2052/2070 (99.13%)

Train Epoch: 147, Average Loss: 0.0215
time 1.5 sec:
Test set: Average loss: 0.0219, Accuracy: 2052/2070 (99.13%)

Train Epoch: 148, Average Loss: 0.0222
time 1.5 sec:
Test set: Average loss: 0.0203, Accuracy: 2052/2070 (99.13%)

Train Epoch: 149, Average Loss: 0.0199
time 1.4 sec:
Test set: Average loss: 0.0244, Accuracy: 2051/2070 (99.08%)

Train Epoch: 150, Average Loss: 0.0180
time 1.4 sec:
Test set: Average loss: 0.0316, Accuracy: 2047/2070 (98.89%)

Train Epoch: 151, Average Loss: 0.0188
time 1.5 sec:
Test set: Average loss: 0.0372, Accuracy: 2044/2070 (98.74%)

Train Epoch: 152, Average Loss: 0.0201
time 1.4 sec:
Test set: Average loss: 0.0203, Accuracy: 2055/2070 (99.28%)

Train Epoch: 153, Average Loss: 0.0179
time 1.5 sec:
Test set: Average loss: 0.0

In [6]:
model.eval()
y_pred = []
y_test = []
with torch.no_grad():
    for data, target in test_loader:
        data = data.cuda()
        output = model(data)
        pred = output.max(1, keepdim=True)[1]  
        pred = pred.cpu().numpy().squeeze()
        y_pred.append(pred)
        y_test.append(target.numpy().squeeze())
y_pred=np.hstack(y_pred)
y_test=np.hstack(y_test)

summary(y_pred,y_test)

confusion matrix:
 [[1828    8]
 [   9  225]]
Roc score: 0.9786
F1 score: 0.9636
Recall_score: 0.9615
Precision_score: 0.9657
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1836
           1       0.97      0.96      0.96       234

    accuracy                           0.99      2070
   macro avg       0.98      0.98      0.98      2070
weighted avg       0.99      0.99      0.99      2070



In [7]:
torch.save(model, '/home/yaso/EQ_Place/code/models/TFEQ_v2.pth')

In [8]:
model.eval()
y_pred = []
out=[]
y_test = []
with torch.no_grad():
    for data, target in test_loader:
        data = data.cuda()
        output = model(data)
        output=F.softmax(output, dim=1)
        pred = output.max(1, keepdim=True)[1]  
        pred = pred.cpu().numpy().squeeze()
        y_pred.append(pred)
        out.append(output.cpu().numpy())
        y_test.append(target.numpy().squeeze())
y_pred=np.hstack(y_pred)
y_test=np.hstack(y_test)
out=np.vstack(out)

summary(y_pred,y_test)

confusion matrix:
 [[1829    7]
 [   8  226]]
Roc score: 0.9810
F1 score: 0.9679
Recall_score: 0.9658
Precision_score: 0.9700
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1836
           1       0.97      0.97      0.97       234

    accuracy                           0.99      2070
   macro avg       0.98      0.98      0.98      2070
weighted avg       0.99      0.99      0.99      2070



In [9]:
earth=np.where(y_pred==1)[0]
non_earth=np.where(y_pred==0)[0]

p_earth=out[earth,1].squeeze()

p_non_earth=out[non_earth,0].squeeze()

In [12]:
a = np.random.rand(100)
hist,bins = np.histogram(p_earth,bins=10,range=(0,1))
print(hist)
print(bins)


[  0   0   0   0   0   1   4   3   9 217]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [13]:
a = np.random.rand(100)
hist,bins = np.histogram(p_non_earth,bins=10,range=(0,1))
print(hist)
print(bins)

[   0    0    0    0    0    6    5    4    5 1816]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
